In [1]:
from dotenv import load_dotenv

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema.document import Document
from langchain import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI


import os
import json
from pprint import pprint

import chromadb
from chromadb.config import Settings

import pandas as pd

from openai import OpenAI
from pathlib import Path


In [2]:
load_dotenv()

DATA_LOCATION = os.environ.get("DATA_LOCATION")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# Challenge I: Erstelle einen Radiobeitrag aus Bestandteilen von SMC-Angeboten

Die Angebote des Science Media Centers enthalten sehr viel Informationen zu Themen mit Wissenschaftsbezug. Hier ein [Beispiel](https://www.sciencemediacenter.de/alle-angebote/research-in-context/details/news/taurin-als-pille-fuer-langlebigkeit/) zur Frage ob Taurin als Pille für Langlebigkeit verwendet werden kann.

Neben längeren redaktionell verfassten Einführungen in ein Thema ("Teaser") sind wissenschaftliche Einordnungen von Expert:innen in Form von Statements zentral für die Angebotsformen "Research in Context" und "Rapid Reaction".

Um den Datensatz kennenzulernen und erste Gehversuche mit der Vektordatenbank ChromaDB und Chains in LangChain zu unternehmen, stellen wir uns heute gemeinsam der Challenge, **einen Radiobeitrag (zu einem Thema) aus Bestandteilen von SMC-Angeboten zu erstellen**.

## 0. Helferfunktionen

In [4]:
def get_dbs():
    """Erzeugt ChromaDB-Client und stellt Verbindung zu zwei Collections her"""
    persistent_client = chromadb.PersistentClient(
        path=f"{DATA_LOCATION}/chroma_db", settings=Settings(allow_reset=True)
    )  # should be created once and passed around

    teaser_db = Chroma(
        client=persistent_client,
        collection_name="story_teaser",
        embedding_function=OpenAIEmbeddings(),
    )

    statement_db = Chroma(
        client=persistent_client,
        collection_name="story_statement",
        embedding_function=OpenAIEmbeddings(),
    )

    return teaser_db, statement_db, persistent_client

In [5]:
def chroma_to_dataframe(chroma_results):
    """Kleiner Helfer, um aus den Ergebnissen von Anfragen gegen ChromaDB DataFrames mit aufgelösten Metadaten zu erzeugen"""
    df = pd.DataFrame()

    if isinstance(chroma_results, dict):
        df = pd.DataFrame(chroma_results)
    elif isinstance(chroma_results, list) and isinstance(chroma_results[0], Document):
        df = pd.DataFrame(map(lambda x: {"documents": x.page_content, "metadatas": x.metadata}, chroma_results))

    if not df.empty:
        metadata = pd.json_normalize(df.metadatas)
        df = pd.concat([df, metadata], axis=1).drop(columns="metadatas")

    return df

## 1. Lade die collections

In [6]:
teaser_db, statement_db, _ = get_dbs()

In [7]:
print("Es gibt", teaser_db._collection.count(), "Dokumente in 'teaser_db' und", statement_db._collection.count(), "Dokumente in 'statement_db'. Dokumente sind in diesem Kontext Teile der Texte und keine vollständige Teaser oder Statements")

Es gibt 2635 Dokumente in 'teaser_db' und 16980 Dokumente in 'statement_db'. Dokumente sind in diesem Kontext Teile der Texte und keine vollständige Teaser oder Statements


## 2. Beispiele für Abfragen

In [8]:
# Filter nach story_no >= 23000
# k = 10 (default = 4) für mehr Ergebnisse

teaser_res = teaser_db.similarity_search(
    query="Was sind die Erklärungen fur den Insektenrückgang",
    k=10,
    filter={"story_no": {"$gte": 23000}}
)

statement_res = statement_db.similarity_search(
    query="Was sind die Erklärungen fur den Insektenrückgang",
    k=10,
    filter={"story_no": {"$gte": 23000}}
)

### Die 10 abgefragten teaser docs sortiert nach story_no und start_index

In [9]:
teaser_df = chroma_to_dataframe(teaser_res).sort_values(by=["story_no", "start_index"])
teaser_df

,documents,publication_date,ressort,source,start_index,story_no,type,url
8,Das Insektensterben ist gut belegt: Globale so...,2023-02-01,Klima & Umwelt,/workspace/data/story_teaser/23020.txt,1347,23020,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
7,Das SMC hat Forschende um eine Einschätzung de...,2023-02-01,Klima & Umwelt,/workspace/data/story_teaser/23020.txt,2209,23020,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
3,Die Studie erscheint vor dem Hintergrund einer...,2023-04-26,Klima & Umwelt,/workspace/data/story_teaser/23064.txt,2018,23064,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
0,Metastudie sucht nach Ursachen für Insektenrüc...,2023-08-23,Klima & Umwelt,/workspace/data/story_teaser/23140.txt,0,23140,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
4,Der seit vielen Jahren intensiv beforschte und...,2023-08-23,Klima & Umwelt,/workspace/data/story_teaser/23140.txt,54,23140,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
9,die Autorinnen und Autoren der Studie 87 spezi...,2023-08-23,Klima & Umwelt,/workspace/data/story_teaser/23140.txt,2216,23140,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
1,Erklärt das Wetter den Insektenrückgang?\n\nDe...,2023-09-27,Klima & Umwelt,/workspace/data/story_teaser/23162.txt,0,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
2,"2017 erschien die Krefeld-Studie, für die 27 J...",2023-09-27,Klima & Umwelt,/workspace/data/story_teaser/23162.txt,878,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
5,Für die nun vorliegende Analyse ergänzten die ...,2023-09-27,Klima & Umwelt,/workspace/data/story_teaser/23162.txt,1358,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
6,"Das SMC hat Expertinnen und Experten befragt, ...",2023-09-27,Klima & Umwelt,/workspace/data/story_teaser/23162.txt,2355,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...


In [33]:
statement_df = chroma_to_dataframe(statement_res)
statement_df

,documents,expert_affiliation,expert_name,publication_date,ressort,source,start_index,statement_no,story_no,type,url
0,Erklärt das Wetter den Insektenrückgang?\n\nDi...,"Professor für Chemische Ökologie, Leiter Fachg...",Prof. Dr. Johannes Steidle,2023-09-27,Klima & Umwelt,/workspace/data/story_statement/23162_2.txt,0,23162_2_0_statement,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
1,Erklärt das Wetter den Insektenrückgang?\n\nUm...,Leiter des Lehrstuhls für Tierökologie und Tro...,Prof. Dr. Ingolf Steffan-Dewenter,2023-09-27,Klima & Umwelt,/workspace/data/story_statement/23162_1.txt,0,23162_1_0_statement,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
2,Erklärt das Wetter den Insektenrückgang?\n\nUn...,außerplanmäßiger Professor im Fachbereich Biog...,Prof. Dr. Axel Hochkirch,2023-09-27,Klima & Umwelt,/workspace/data/story_statement/23162_7.txt,0,23162_7_0_statement,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
3,Es wurde bereits von den Autoren der Krefeld-S...,Leiter der Arbeitsgruppe Tierökologie und mult...,Prof. Dr. Christoph Scherber,2023-09-27,Klima & Umwelt,/workspace/data/story_statement/23162_3.txt,1589,23162_3_2_statement,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
4,Trotz der oben angesprochenen Mängel deckt sic...,"Professor für Chemische Ökologie, Leiter Fachg...",Prof. Dr. Johannes Steidle,2023-08-23,Klima & Umwelt,/workspace/data/story_statement/23140_0.txt,3372,23140_0_5_statement,23140,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
5,Methodik\n\nDie Arbeit befasst sich mit der Fr...,"Professor für Chemische Ökologie, Leiter Fachg...",Prof. Dr. Johannes Steidle,2023-09-27,Klima & Umwelt,/workspace/data/story_statement/23162_2.txt,729,23162_2_1_statement,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
6,Metastudie sucht nach Ursachen für Insektenrüc...,"Professor für Chemische Ökologie, Leiter Fachg...",Prof. Dr. Johannes Steidle,2023-08-23,Klima & Umwelt,/workspace/data/story_statement/23140_0.txt,0,23140_0_0_statement,23140,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
7,Metastudie sucht nach Ursachen für Insektenrüc...,Professorin für Naturschutz und Landschaftsöko...,Prof. Dr. Alexandra-Maria Klein,2023-08-23,Klima & Umwelt,/workspace/data/story_statement/23140_2.txt,0,23140_2_0_statement,23140,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
8,"Auf die Frage, inwiefern die Schlussfolgerung ...",Leiter des Lehrstuhls für Tierökologie und Tro...,Prof. Dr. Ingolf Steffan-Dewenter,2023-09-27,Klima & Umwelt,/workspace/data/story_statement/23162_1.txt,1917,23162_1_3_statement,23162,Research in Context,https://www.sciencemediacenter.de/alle-angebot...
9,Metastudie sucht nach Ursachen für Insektenrüc...,"Postdoc am sDiv – Synthesezentrum, Deutsches Z...",Dr. Roel van Klink,2023-08-23,Klima & Umwelt,/workspace/data/story_statement/23140_1.txt,0,23140_1_0_statement,23140,Research in Context,https://www.sciencemediacenter.de/alle-angebot...


### Die 10 abgefragten statement docs gruppiert nach expert_name und story_no, sortiert nach start_index

In [43]:
grouped_by_expert = (statement_df
              .sort_values(by=["expert_name", "expert_affiliation", "story_no", "start_index"])
              .groupby(by=["expert_name", "expert_affiliation", "story_no"])
              [["documents", "start_index"]]
              .agg(list)
              .explode(["start_index", "documents"])
             )
grouped_by_expert

documents  \
expert_name                       expert_affiliation                                 story_no                                                      
Dr. Roel van Klink                Postdoc am sDiv – Synthesezentrum, Deutsches Ze... 23140     Metastudie sucht nach Ursachen für Insektenrüc...   
Prof. Dr.  Christoph Scherber     Leiter der Arbeitsgruppe Tierökologie und multi... 23162     Es wurde bereits von den Autoren der Krefeld-S...   
Prof. Dr. Alexandra-Maria Klein   Professorin für Naturschutz und Landschaftsökol... 23140     Metastudie sucht nach Ursachen für Insektenrüc...   
Prof. Dr. Axel Hochkirch          außerplanmäßiger Professor im Fachbereich Bioge... 23162     Erklärt das Wetter den Insektenrückgang?\n\nUn...   
Prof. Dr. Ingolf Steffan-Dewenter Leiter des Lehrstuhls für Tierökologie und Trop... 23162     Erklärt das Wetter den Insektenrückgang?\n\nUm...   
                                                                                     23162     Auf die Frage, inwiefern die Schlussfolgerung ...   
Prof. Dr. Johannes Steidle        Professor für Chemische Ökologie, Leiter Fachge... 23140     Metastudie sucht nach Ursachen für Insektenrüc...   
                                                                                     23140     Trotz der oben angesprochenen Mängel deckt sic...   
                                                                                     23162     Erklärt das Wetter den Insektenrückgang?\n\nDi...   
                                                                                     23162     Methodik\n\nDie Arbeit befasst sich mit der Fr...   

                                                                                              start_index  
expert_name                       expert_affiliation                                 story_no              
Dr. Roel van Klink                Postdoc am sDiv – Synthesezentrum, Deutsches Ze... 23140              0  
Prof. Dr.  Christoph Scherber     Leiter der Arbeitsgruppe Tierökologie und multi... 23162           1589  
Prof. Dr. Alexandra-Maria Klein   Professorin für Naturschutz und Landschaftsökol... 23140              0  
Prof. Dr. Axel Hochkirch          außerplanmäßiger Professor im Fachbereich Bioge... 23162              0  
Prof. Dr. Ingolf Steffan-Dewenter Leiter des Lehrstuhls für Tierökologie und Trop... 23162              0  
                                                                                     23162           1917  
Prof. Dr. Johannes Steidle        Professor für Chemische Ökologie, Leiter Fachge... 23140              0  
                                                                                     23140           3372  
                                                                                     23162              0  
                                                                                     23162            729

## Eure Arbeit

...

## Text to speech

In [12]:
client = OpenAI()


output_file_path = Path(os.path.abspath('')).parent / "results" 


In [15]:
radio_report = """Eine bahnbrechende Studie hat ergeben, dass die Menge der Aminosäure Taurin im Körper von Menschen und Tieren mit dem Alter abnimmt und dass dieser Verlust sogar zum Altern beitragen könnte. Die Forscher fanden heraus, dass eine Supplementierung mit Taurin den Alterungsprozess möglicherweise umkehren könnte. Unabhängige Wissenschaftler sind von den Ergebnissen überzeugt, warnen jedoch davor, Selbstversuche ohne klinische Studien durchzuführen. 
Dr. Sebastian Grönke vom Max-Planck-Institut für Biologie des Alterns in Köln sieht in der Studie vielversprechende Ergebnisse. Er betont, dass Taurin bereits in früheren Studien mit einer verlängerten Lebensdauer in Verbindung gebracht wurde. Allerdings warnt er auch vor voreiligen Schlüssen und betont die Notwendigkeit weiterer klinischer Studien.
Dr. Clara Correia-Melo vom Leibniz-Institut für Alternsforschung lobt die Qualität der Studie und hebt hervor, dass die Autoren nicht nur den Zusammenhang zwischen Taurin und Alterung untersucht haben, sondern auch die Kausalität. Sie sieht jedoch noch Forschungsbedarf, insbesondere in Bezug auf die Interaktion von Taurin mit anderen Nährstoffen.
Professorin Dr. Kristina Norman vom Deutschen Institut für Ernährungsforschung warnt davor, voreilige Schlüsse zu ziehen und betont, dass es unwahrscheinlich sei, dass ein einzelner Nährstoff den komplexen Alterungsprozess allein beeinflussen könne.
Zusammenfassend lässt sich sagen, dass die Studie spannende Ergebnisse liefert, aber noch viele Fragen offenlässt. Es bleibt abzuwarten, was zukünftige Forschungen ergeben werden. Bleiben Sie dran, um auf dem Laufenden zu bleiben."""

In [16]:
response = client.audio.speech.create(
  model="tts-1",
  voice="nova",
  input=radio_report
)

response.stream_to_file(output_file_path / "radio_report_example.mp3")